In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
import cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array

### 데이터 분석

In [2]:
IX = 80
IY = 80

In [3]:
from glob import glob
root='c:/data/kfood'
fileName,superDir,subDir=[],[],[]
imgList = []
invalid_imgList=[]
for spd in glob(root + '/*'):  # 국, 구이등 대분류
     for sbd in glob(spd + '/*'): # 구이중 -> 갈치구이, 고등어구이
        for file in glob(sbd + '/*.jpg'):  # 파일:  d:/kfood\\구이\\갈치구이\\Img_001_0010.jpg 이 형식임으로 split로 나누어서
                                               #      ['d:/kfood', '국', '무국', 'Img_016_0951.jpg']
            splitFile=file.split('\\')
           
            fileName.append(splitFile[3])
            superDir.append(splitFile[1])
            subDir.append(splitFile[2])
            try:
               mapFile = '/'.join(splitFile)
               image = cv2.imread(mapFile)
               image = cv2.resize(image, (IX,IY) )
               image = img_to_array(image)
               imgList.append(image)
            except:
               invalid_imgList.append(splitFile[3])

df=pd.DataFrame({'fileName':fileName,'superDir':superDir,'subDir':subDir})
df.head()

,fileName,superDir,subDir
0,Img_000_0000.jpg,roast,galbi-roast
1,Img_000_0001.jpg,roast,galbi-roast
2,Img_000_0002.jpg,roast,galbi-roast
3,Img_000_0003.jpg,roast,galbi-roast
4,Img_000_0004.jpg,roast,galbi-roast


In [4]:
print('invalid ids:')
print(invalid_imgList)
invList = pd.DataFrame(invalid_imgList, columns=['fileName'])
invList

invalid ids:
['Img_000_0145.jpg', 'Img_000_0197.jpg', 'Img_000_0215.jpg', 'Img_000_0253.jpg', 'Img_001_0067.jpg', 'Img_001_0286.jpg', 'Img_001_0355.jpg', 'Img_015_0294.jpg', 'Img_015_0320.jpg', 'Img_015_0503.jpg']


,fileName
0,Img_000_0145.jpg
1,Img_000_0197.jpg
2,Img_000_0215.jpg
3,Img_000_0253.jpg
4,Img_001_0067.jpg
5,Img_001_0286.jpg
6,Img_001_0355.jpg
7,Img_015_0294.jpg
8,Img_015_0320.jpg
9,Img_015_0503.jpg


In [5]:
df

,fileName,superDir,subDir
0,Img_000_0000.jpg,roast,galbi-roast
1,Img_000_0001.jpg,roast,galbi-roast
2,Img_000_0002.jpg,roast,galbi-roast
3,Img_000_0003.jpg,roast,galbi-roast
4,Img_000_0004.jpg,roast,galbi-roast
...,...,...,...
6779,Img_016_0993.jpg,soup,moo-soup
6780,Img_016_0994.jpg,soup,moo-soup
6781,Img_016_0997.jpg,soup,moo-soup
6782,Img_016_0998.jpg,soup,moo-soup


In [6]:
np.shape(imgList)

(6774, 80, 80, 3)

In [7]:
labels = []

used_columns = ['subDir','superDir']

# getting labels for the columns used

for index, row in df.iterrows():

    if row['fileName'] in invalid_imgList:
        continue

    tags = []

    for col in used_columns:
        tags.append(row[col])

    labels.append(tags)

labels

[['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 ['galbi-roast', 'roast'],
 

In [8]:
import numpy as np

# converting data into numpy arrays

data = np.array(imgList, dtype="float") / 255.0
labels = np.array(labels)

print(labels)

[['galbi-roast' 'roast']
 ['galbi-roast' 'roast']
 ['galbi-roast' 'roast']
 ...
 ['moo-soup' 'soup']
 ['moo-soup' 'soup']
 ['moo-soup' 'soup']]


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

# creating a binary vector for the input labels 

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

print(mlb.classes_)
print(labels[0])

['egg-soup' 'galbi-roast' 'galchi-roast' 'gobchang-roast'
 'godeunguh-roast' 'mando-soup' 'moo-soup' 'roast' 'soup']
[0 1 0 0 0 0 0 1 0]


In [10]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

inputShape = (IY, IX, 3)

# A very simple sequential model is used since the images are very low resolution and the categories are fiarly distinct

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

model.add(Flatten()) 

model.add(Dense(128))
model.add(Activation('sigmoid'))


out = len(mlb.classes_)

model.add(Dense(out))
model.add(Activation('sigmoid')) # activation function for the final layer has to be sigmoid, since mutiple output labels can have value 1
                    
model.compile(loss='binary_crossentropy', # loss function has to be binary_crossentropy, it is calculated seperately for each of the outputs
              optimizer='adam',
              metrics=['mse'])

In [11]:
from sklearn.model_selection import train_test_split

# splitting data into testing and training set 

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.1, random_state=42)

In [12]:
batch = 32
E = 25

#training the model 
model.fit(x=trainX,y=trainY,
          epochs=E ,verbose=1)

Epoch 1/25
191/191 [==============================] - 11s 56ms/step - loss: 0.4131 - mse: 0.1280
Epoch 2/25
191/191 [==============================] - 11s 59ms/step - loss: 0.3276 - mse: 0.1003
Epoch 3/25
191/191 [==============================] - 12s 61ms/step - loss: 0.2673 - mse: 0.0806
Epoch 4/25
191/191 [==============================] - 12s 61ms/step - loss: 0.2054 - mse: 0.0595
Epoch 5/25
191/191 [==============================] - 12s 62ms/step - loss: 0.1527 - mse: 0.0418
Epoch 6/25
191/191 [==============================] - 12s 61ms/step - loss: 0.1041 - mse: 0.0258
Epoch 7/25
191/191 [==============================] - 12s 61ms/step - loss: 0.0686 - mse: 0.0148
Epoch 8/25
191/191 [==============================] - 12s 61ms/step - loss: 0.0445 - mse: 0.0081
Epoch 9/25
191/191 [==============================] - 12s 61ms/step - loss: 0.0285 - mse: 0.0040
Epoch 10/25
191/191 [==============================] - 12s 61ms/step - loss: 0.0193 - mse: 0.0022
Epoch 11/25
191/191 [========

In [13]:
preds = model.predict(testX)


# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized = []

for pred in preds:
    vals = []
    for val in pred:
        if val > 0.5:
            vals.append(1)
        else:
            vals.append(0)
    pred_binarized.append(vals) 

pred_binarized = np.array(pred_binarized)   


# we convert the output vectors to the predicted labels
true_test_labels = mlb.inverse_transform(testY)
pred_test_labels = mlb.inverse_transform(pred_binarized)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(testY)):

    true_labels = list(true_test_labels[i])

    pred_labels = list(pred_test_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]

    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    



print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

22/22 [==============================] - 0s 9ms/step
correct:  921
missing/wrong:  435
Accuracy:  0.6792035398230089


In [14]:
for i in range(20):
    print('True labels: ',true_test_labels[i],' Predicted labels: ',pred_test_labels[i])

True labels:  ('gobchang-roast', 'roast')  Predicted labels:  ('gobchang-roast', 'roast')
True labels:  ('moo-soup', 'soup')  Predicted labels:  ('moo-soup', 'soup')
True labels:  ('galchi-roast', 'roast')  Predicted labels:  ('roast',)
True labels:  ('galbi-roast', 'roast')  Predicted labels:  ('galbi-roast', 'roast')
True labels:  ('galchi-roast', 'roast')  Predicted labels:  ('gobchang-roast', 'roast')
True labels:  ('gobchang-roast', 'roast')  Predicted labels:  ('gobchang-roast', 'roast')
True labels:  ('galchi-roast', 'roast')  Predicted labels:  ('roast',)
True labels:  ('galchi-roast', 'roast')  Predicted labels:  ('galchi-roast', 'roast')
True labels:  ('galbi-roast', 'roast')  Predicted labels:  ('godeunguh-roast', 'roast')
True labels:  ('godeunguh-roast', 'roast')  Predicted labels:  ('roast',)
True labels:  ('godeunguh-roast', 'roast')  Predicted labels:  ('galbi-roast', 'roast')
True labels:  ('gobchang-roast', 'roast')  Predicted labels:  ('galbi-roast', 'roast')
True la

In [47]:
from PIL import Image
import numpy as np
img=Image.open('c:/data/kfood-test/galbi3.jpg').resize((IX,IY)).convert('RGB')
img=np.array(img)
img=np.reshape(img, (1,IX,IY,3))
result=model.predict(img)
print(result)

1/1 [==============================] - 0s 15ms/step
[[1.8720354e-01 6.7471950e-07 2.6678541e-03 7.0883136e-05 1.3595179e-01
  8.7266020e-02 6.5439560e-02 1.7213746e-03 9.9634659e-01]]
